In [ ]:
# STEP 0 – CLEAN UP & DOWNLOAD ALL DATASETS INTO THIS FOLDER (run once)
# -------------------------------------------------------------------
# • Deletes old ~/gun-detection-yolo folder
# • Deletes any prior dataset1/, dataset2/, roboflow_pistols/ in PWD
# • Downloads Kaggle datasets into ./dataset1 & ./dataset2
# • Extracts your roboflow_pistols.zip into ./roboflow_pistols

# 0-a) install & import
import os, shutil, zipfile, glob
from pathlib import Path
from kaggle.api.kaggle_api_extended import KaggleApi

# 0-b) remove old root downloads (~/.…/gun-detection-yolo)
old_root = Path.home() / "gun-detection-yolo"
if old_root.exists():
    shutil.rmtree(old_root)
    print(f"🗑  Removed old root folder: {old_root}")

# 0-c) remove any leftover folders in PWD
pwd = Path.cwd()
for name in ("dataset1", "dataset2", "roboflow_pistols"):
    p = pwd / name
    if p.exists():
        if p.is_dir():
            shutil.rmtree(p)
        else:
            p.unlink()
        print(f"🗑  Removed existing: {p}")

# 0-d) authenticate Kaggle API
api = KaggleApi()
api.authenticate()

# 0-e) download & unzip dataset1 → ./dataset1
print("⇣ Downloading issaisasank/guns-object-detection → dataset1/")
api.dataset_download_files(
    "issaisasank/guns-object-detection",
    path=str(pwd / "dataset1"),
    unzip=True
)
print("✔ dataset1 ready at", pwd / "dataset1")

# 0-f) download & unzip dataset2 → ./dataset2
print("⇣ Downloading snehilsanyal/weapon-detection-test → dataset2/")
api.dataset_download_files(
    "snehilsanyal/weapon-detection-test",
    path=str(pwd / "dataset2"),
    unzip=True
)
print("✔ dataset2 ready at", pwd / "dataset2")

# 0-g) extract roboflow_pistols.zip → ./roboflow_pistols
rf_zip = pwd / "roboflow_pistols.zip"
if not rf_zip.is_file():
    raise FileNotFoundError(f"❌ {rf_zip.name} not found in {pwd}")
print("⇣ Extracting Roboflow pistols → roboflow_pistols/")
with zipfile.ZipFile(rf_zip) as zf:
    zf.extractall(pwd / "roboflow_pistols")
print("✔ roboflow_pistols ready at", pwd / "roboflow_pistols")

print("\n✅  Cleanup & downloads complete. Datasets are in:\n",
      pwd / "dataset1", "\n",
      pwd / "dataset2", "\n",
      pwd / "roboflow_pistols")

In [ ]:
# STEP 2 – merge dataset1, dataset2, roboflow_pistols → ./data
# ------------------------------------------------------------
from pathlib import Path
import shutil, random, yaml, sys, itertools

cwd = Path.cwd()

# 2-a)  👉  define roots & auto-detect images / labels
roots = {
    "dataset1": cwd / "dataset1",
    "dataset2": cwd / "dataset2",
    "pistols":  cwd / "roboflow_pistols" / "export",
}
for k, p in roots.items():
    if not p.exists():
        sys.exit(f"❌ {k} root not found at {p}. Check Step 1 output.")

def find_subdirs(root):
    """return (img_dir, lbl_dir) inside root"""
    # Look for direct images/labels
    img = next((d for d in root.rglob("*") if d.name.lower() in ("images", "imgs")), None)
    lbl = next((d for d in root.rglob("*") if d.name.lower().startswith("label")), None)
    return img, lbl

sources = []
for tag, root in roots.items():
    img_dir, lbl_dir = find_subdirs(root)
    if not (img_dir and lbl_dir):
        sys.exit(f"❌ Couldn’t find images/labels under {root}")
    sources.append((tag, img_dir, lbl_dir))

# 2-b)  👉  fresh YOLO skeleton
yolo_root = cwd / "data"
if yolo_root.exists():
    shutil.rmtree(yolo_root)
for split in ("train", "val"):
    (yolo_root / split / "images").mkdir(parents=True, exist_ok=True)
    (yolo_root / split / "labels").mkdir(parents=True, exist_ok=True)

# 2-c)  👉  gather all pairs, prefix names to avoid collisions
pairs = []
image_exts = {".jpg", ".jpeg", ".png"}
for tag, img_dir, lbl_dir in sources:
    for img in img_dir.glob("*"):
        if img.suffix.lower() not in image_exts:
            continue
        lbl = lbl_dir / f"{img.stem}.txt"
        if lbl.exists():
            new_stem = f"{tag}_{img.stem}"
            pairs.append((img, lbl, new_stem))

random.seed(42)
random.shuffle(pairs)
cut = int(0.8 * len(pairs))

def copy_subset(subset, split):
    for img, lbl, stem in subset:
        shutil.copy2(img,  yolo_root / split / "images" / f"{stem}{img.suffix.lower()}")
        shutil.copy2(lbl,  yolo_root / split / "labels" / f"{stem}.txt")

copy_subset(pairs[:cut], "train")
copy_subset(pairs[cut:], "val")

# 2-d)  👉  write data.yaml
yaml.safe_dump({
    "train": str((yolo_root / "train" / "images").resolve()),
    "val":   str((yolo_root / "val"   / "images").resolve()),
    "nc": 1,
    "names": ["gun"]
}, open(yolo_root / "data.yaml", "w"))

print(f"✔ Merged {len(pairs)} images  ➜  {len(pairs[:cut])} train / {len(pairs[cut:])} val")
print("✔ YOLO data.yaml written to", (yolo_root / "data.yaml").resolve())

In [ ]:
# STEP 3 – heavy augmentation for robust gun detection
# ----------------------------------------------------
!pip -q install --upgrade albumentations opencv-python-headless

from pathlib import Path
import albumentations as A
import cv2, uuid
from tqdm.auto import tqdm

train_img_dir   = Path("data/train/images")
train_label_dir = Path("data/train/labels")

# Bbox helper (YOLO format)
bbox_args = A.BboxParams(format="yolo", label_fields=["class_ids"])

augs = {
    "flip"      : A.Compose([A.HorizontalFlip(always_apply=True)], bbox_params=bbox_args),
    "gray"      : A.Compose([A.ToGray(always_apply=True)], bbox_params=bbox_args),
    "rgbshift"  : A.Compose([A.RGBShift(25,25,25, always_apply=True)], bbox_params=bbox_args),
    "bright"    : A.Compose([A.RandomBrightnessContrast(0.25,0.25, always_apply=True)], bbox_params=bbox_args),
    "blur"      : A.Compose([A.MotionBlur(blur_limit=5, always_apply=True)], bbox_params=bbox_args),
    "fog"       : A.Compose([A.RandomFog(fog_coef_lower=0.1, fog_coef_upper=0.3,
                                         alpha_coef=0.08, always_apply=True)], bbox_params=bbox_args),
}

def load_yolo(txt):
    rows=[]
    with open(txt) as f:
        for line in f:
            parts=line.strip().split()
            if len(parts)<5: continue
            cls=int(parts[0]); box=list(map(float,parts[1:5]))
            rows.append([cls]+box)
    return rows

def save_yolo(txt,rows):
    with open(txt,"w") as f:
        for r in rows:
            cls,*box=r
            f.write(f"{cls} "+" ".join(f"{v:.6f}" for v in box)+"\n")

img_exts={".jpg",".jpeg",".png"}
aug_cnt=0

for img_path in tqdm(list(train_img_dir.iterdir()), desc="Augmenting"):
    if img_path.suffix.lower() not in img_exts: continue
    lbl_path=train_label_dir / f"{img_path.stem}.txt"
    if not lbl_path.exists(): continue

    rows=load_yolo(lbl_path)
    if not rows: continue

    image=cv2.imread(str(img_path))
    bboxes=[r[1:] for r in rows]
    class_ids=[r[0]   for r in rows]

    for tag,aug in augs.items():
        res=aug(image=image,bboxes=bboxes,class_ids=class_ids)
        if not res["bboxes"]: continue

        uid=uuid.uuid4().hex[:6]
        new_name=f"{img_path.stem}_{tag}_{uid}"
        cv2.imwrite(str(train_img_dir/f"{new_name}{img_path.suffix.lower()}"), res["image"])

        new_rows=[[c,*b] for c,b in zip(res["class_ids"], res["bboxes"])]
        save_yolo(train_label_dir/f"{new_name}.txt", new_rows)
        aug_cnt+=1

print(f"✔ Augmentation complete – created {aug_cnt} new images in {train_img_dir}")

In [18]:
# STEP 4 – YOLO-v8 NANO  ·  memory-safe on M-series
# -------------------------------------------------
from ultralytics import YOLO
import torch, warnings, multiprocessing as mp

device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"🖥  Training on ➜ {device}")

model = YOLO("yolov8n.pt")              # 3 M params

base_args = dict(
    data="data/data.yaml",
    epochs=100,
    imgsz=640,           # ↓ smaller, still fine for pistols
    device=device,
    workers=mp.cpu_count(),
    amp=True,
    cos_lr=True,
    close_mosaic=20,
    pretrained=True,
    patience=20,
    project="runs",
    name="gun_yolov8n_640",
    verbose=False,
    cache="disk",        # mmap ↔ no giant RAM grab
)

def train_try(b):
    try:
        print(f"\n🚀  Launching training with batch = {b}")
        model.train(batch=b, **base_args)
        return True
    except RuntimeError as e:
        if "out of memory" in str(e).lower():
            warnings.warn(f"⚠️  OOM at batch {b} – trying smaller …")
            torch.mps.empty_cache()
            return False
        else:
            raise

for bs in (8, 4, 2, 1):
    if train_try(bs):
        break
else:
    raise RuntimeError("❌  Even batch=1 fails – free GPU apps or lower imgsz")

print("\n🏁  Training complete!  Best weights ➜ runs/gun_yolov8n_640/weights/best.pt")

🖥  Training on ➜ mps

🚀  Launching training with batch = 8
New https://pypi.org/project/ultralytics/8.3.158 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.156 🚀 Python-3.12.9 torch-2.7.1 MPS (Apple M4 Pro)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=20, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=data/data.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=gun_yolov8n_6402, nbs=64,

train: Scanning /Users/amogharya/Documents/gun detection multiple ds/data/train/labels.cache... 20134 images, 0 backgrounds, 1101 corrupt: 100%|██████████| 20134/20134 [00:00<?, ?it/s]

train: /Users/amogharya/Documents/gun detection multiple ds/data/train/images/dataset1_1.jpeg: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.
train: /Users/amogharya/Documents/gun detection multiple ds/data/train/images/dataset1_10.jpeg: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.
train: /Users/amogharya/Documents/gun detection multiple ds/data/train/images/dataset1_101.jpeg: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.
train: /Users/amogharya/Documents/gun detection multiple ds/data/train/images/dataset1_102.jpeg: ignoring corrupt image/label: setting an array e


train: Caching images (18.3GB Disk): 100%|██████████| 19033/19033 [00:00<00:00, 60409.59it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 128.8±125.2 MB/s, size: 18.7 KB)


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 42/42 [14:27<00:00, 20.66s/it]

                   all        668        837      0.669      0.257      0.257      0.178



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      2/100       4.3G     0.9645       1.21      1.368         21        640:  89%|████████▉ | 2130/2380 [8:46:22<1:01:46, 14.83s/it]  


KeyboardInterrupt: 